In [16]:
import pandas as pd
import geopy.distance

In [17]:
min_population = 100000

df = pd.read_excel('data/geonames-all-cities-with-a-population-1000.xlsx')
df = df[['Name', 'Coordinates', 'Population']]
df['Latitude'] = df['Coordinates'].apply(lambda x: x.split(',')[0]).astype(float)
df['Longitude'] = df['Coordinates'].apply(lambda x: x.split(',')[1]).astype(float)
df = df.drop('Coordinates', axis=1)
df = df.sort_values(by=['Population'], ascending=False)
df = df[df['Population'] > min_population]
df

,Name,Population,Latitude,Longitude
920,Amsterdam,741636,52.37403,4.88969
1390,Rotterdam,598199,51.92250,4.47917
500,The Hague,474292,52.07667,4.29861
1138,Utrecht,361742,52.09083,5.12222
958,Groningen,233218,53.21917,6.56667
1125,Eindhoven,209620,51.44083,5.47778
385,Tilburg,199613,51.55551,5.09130
535,Almere Stad,176432,52.37025,5.21413
1223,Breda,167673,51.58656,4.77596
746,Nijmegen,158732,51.84250,5.85278


In [18]:
dataset = pd.read_csv('data/dataset.csv')
dataset['closest_city'] = ''
dataset['distance_to_center'] = 1e6
dataset['population_of_closest_city'] = 0
dataset.head()

,funda_identifier,location_part1,location_part2,location_part3,price,living_area,plot_size,bedrooms,asking_price_per_m²,listed_since,...,bedroom,type_apartment,located_at,balcony/roof_garden,accessibility,specific,quality_marks,closest_city,distance_to_center,population_of_closest_city
0,42126054,Hogeweg 66,3814 CE Amersfoort,Kruiskamp-Zuid,450000,110,189,2,4.091,Today,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1000000.0,0
1,42126250,Olivijn 8,5912 PV Venlo,Hagerbroek,498000,160,333,4,3.112,Today,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1000000.0,0
2,42189920,Dwarslaan 10,1261 BB Blaricum,Blaricum-Dorp,1750000,206,1153,3,8.495,"May 12, 2023",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1000000.0,0
3,42115579,Vuurdoornstraat 1,6101 MB Echt,Echt Noord,399000,157,419,4,2.541,"May 14, 2023",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1000000.0,0
4,42106112,Van Lennephof 1,5343 JC Oss,Vondellaan-Zuid,300000,124,205,4,2.419,"May 8, 2023",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1000000.0,0


In [19]:
def find_closest_city(lat, lon):
    min_distance = 1e6
    for _, row in df.iterrows():
        distance = geopy.distance.distance((lat, lon), (row['Latitude'], row['Longitude'])).km
        if distance < min_distance:
            min_distance = distance
            closest_city = row['Name']
            population = row['Population']
    return closest_city, min_distance, population

for index, row in dataset.iterrows():
    lat = row['lat']
    lon = row['lon']
    closest_city, distance, population = find_closest_city(lat, lon)
    dataset.at[index, 'closest_city'] = closest_city
    dataset.at[index, 'distance_to_center'] = distance
    dataset.at[index, 'population_of_closest_city'] = population


In [20]:
dataset[['closest_city', 'population_of_closest_city', 'distance_to_center', 'lat', 'lon']].head(20)

,closest_city,population_of_closest_city,distance_to_center,lat,lon
0,Amersfoort,139914,0.900081,52.15718,5.400165
1,Venlo,101603,1.769002,51.35439,6.172892
2,Almere Stad,176432,10.208437,52.27979,5.239072
3,Maastricht,122378,32.186529,51.11259,5.875482
4,'s-Hertogenbosch,134520,15.223303,51.74699,5.510582
5,Zoetermeer,115845,10.520475,52.03252,4.640951
6,Amsterdam,741636,2.737053,52.36955,4.929205
7,Tilburg,199613,10.488207,51.54571,4.940913
8,Breda,167673,85.625491,51.31906,3.620937
9,'s-Hertogenbosch,134520,19.013661,51.86996,5.294795


In [21]:
dataset[['closest_city', 'population_of_closest_city', 'distance_to_center', 'lat', 'lon']].sort_values(by=['distance_to_center'], ascending=False).head(20)

,closest_city,population_of_closest_city,distance_to_center,lat,lon
3914,Breda,167673,101.490806,51.30607,3.386818
6860,Breda,167673,101.298579,51.30992,3.387686
14545,Breda,167673,99.716311,51.24596,3.450092
9839,Breda,167673,98.413587,51.26948,3.454958
14501,Rotterdam,598199,97.109807,51.37600,3.385160
5999,Rotterdam,598199,96.428712,51.36963,3.406285
10086,Rotterdam,598199,96.276283,51.36664,3.413191
15152,Zaanstad,140085,95.435813,53.29632,5.072217
4628,Zaanstad,140085,95.107559,53.29449,5.061603
16376,Breda,167673,94.578972,51.32625,3.480585


In [31]:
dataset.to_csv('data/dataset.csv', index=False)